# Attention Mechanism

[Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) is also known as the additive attention. Attention is an extension to encoder-decoder model which learns to align and translate jointly. Each time the model generates a word in a translation, it searches for a set of positions in a source sentence where the most relevant information is concentrated. The model then predicts a target word based on the context vectors associated with these source positions and all the previous generated target words.

The models that came before this idea tried to encode the whole input sentence into a single fixed-length vector. The new attention mechanism encodes the input sentence into a sequence of vectors and chooses only a subset of these vectors adaptively during decoding phase.

## RNN Encoder-Decoder

Suppose we have a RNN based encoder-decoder network, the encoder reads the input sentence, a sequence of vectors
$x = (x_1, x_2, ..., x_{T})$ into a context vector $c$. For every step in the encoding phase, the network generates a hidden state vector at step `t`.

$$
h_t = f\,(x_t, h_{t-1})
$$

The context vector is then generated from the sequence of hidden states.

$$
c = q\,({h_1, h_2, ..., h_T})
$$

$f$ and $q$ are some nonlinear functions that represent the network. The decoder is trained to predict the next word $y_t$ given the context vector and all the previously predicted words $(y_1, y_2, ..., y_{t-1})$. In other words, the decoder defines a probability over the translation $y$ by decomposing the joint probability into the ordered conditionals.

$$
p\,(y) = \prod_{t=1}^T p\,(y_t \mid y_1, y_2, ..., y_{t-1}, c)
$$

The $y$ represents the whole translated sentence. Each conditional probability is modeled as

$$
p\,(y_t \mid y_1, y_2, ..., y_{t-1}, c) = g\,(y_{t-1}, s_t, c)
$$

where $s_t$ is the hidden state of the RNN network.



## Learning to Align

The paper proposes a new model architecture where it uses a bi-directional RNN as an encoder/decoder. Each conditional probability is defined as:

$$
p\,(y_i \mid y_1, y_2, ..., y_{i-1}, x) = g\,(y_{i-1}, s_{i}, c_{i})
$$

$s_i$ is the hidden state at time `i` which is computed by

$$
s_i = f\,(s_{i-1}, y_{i-1}, c_i)
$$

Notice that the probability is conditioned on a **distinct** context vector $c_i$ for each target word output. The context vector depends on a sequence of annotations $(h_1, h_2, ..., h_T)$ to which an encoder maps the input sentence. Each annotation $h_i$ contains information about the whole input sequence with a strong focus on the parts surrounding the `i`th word of the input sequence. These are effectively the hidden states of the encoder at time step `t`.

The context vector is a weighted sum of these annotations.

$$
c_i = \Sigma_{j=1}^{T} \alpha_{ij}h_j
$$

The weight $\alpha_{ij}$ for each annotation $h_j$ is computed by a softmax function.

$$
\alpha_{ij} = \frac{\text{exp}\,(e_{ij})}{ \Sigma_{k=1}^T \text{exp}\, (e_{ik})}
$$

where

$$
e_{ij} = a\,(s_{i-1}, h_j)
$$

The $a$ is called an alignment model which scores ho wwell the inputs around position `j` and the output at position `i` match. $a$ is modeled as a feedforward fully connected neural network which is jointly trained with all the other components of the system.

> So why use a bi-directional RNN?

Annotation of each word should summarize not only the preceding words, but also the following words. The bi-directional allows such information to propagate from left to right and right to left.

The forward RNN reads the input sequences from left to right and calculates a forward hidden state sequence. The backward RNN reads the input sequences from right to left and calculates a backward hidden state sequence. The annotation for each word is the concatenation of the forward and backward hidden states.

> Why is it additive?

Because the context vector is a weighted sum of annotations.

![Bahdanau Attention Model](assets/bahdanau.png)


## Encoder and Decoder in PyTorch

Source: [Practical PyTorch: Translation with a Sequence to Sequence Network and Attention](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb)

In [1]:
import torch
import torch.nn as nn


class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        
    def forward(self, word_inputs, hidden):
        # Note: we run this all at once (over the whole input sequence)
        seq_len = len(word_inputs)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        if USE_CUDA: hidden = hidden.cuda()
        return hidden

In [2]:
class BahdanauAttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        
        # Define parameters
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.attn = GeneralAttn(hidden_size) # Attention unit should compute the softmaxed weights aka alpha.
        self.gru = nn.GRU(hidden_size * 2, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, word_input, last_hidden, encoder_outputs):
        # Note that we will only be running forward for a single decoder time step, but will use all
        # encoder outputs
        
        # Get the embedding of the current input word (last output word)
        word_embedded = self.embedding(word_input).view(1, 1, -1) # S=1 x B x N
        word_embedded = self.dropout(word_embedded)
        
        # Calculate attention weights and apply to encoder outputs (which are the hidden states h)
        attn_weights = self.attn(last_hidden[-1], encoder_outputs) 
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N 
        
        # Combine embedded input word and attended context, run through RNN
        rnn_input = torch.cat((word_embedded, context), 2)
        output, hidden = self.gru(rnn_input, last_hidden)
        
        # Final output layer
        output = output.squeeze(0) # B x N
        output = F.log_softmax(self.out(torch.cat((output, context), 1)))
        
        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights